In [1]:
!pip install catboost
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 92.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 10.1 MB/

In [ ]:
# importation des biblithèques
import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split

from catboost import CatBoostClassifier, Pool

from sklearn.metrics import classification_report


In [ ]:
# importation des données
data = pd.read_excel('/content/drive/Othercomputers/Mon ordinateur/DIT MASTER1 2022/MASTER 2/MEMOIRE/data/data/dataset_cleaned_statusCode_12022024.xlsx')

In [ ]:
data = data.drop('createdAt', axis=1)

In [ ]:
data.head()

,process,amount,phoneNumber,statusCode,canal,country,event,browser,ip,agregator
0,MOBILE,100,22952734444,INSUFFICIENT_BALANCE,Website,BJ,645a225c4744ee6479066772,Light Browser,104.28.217.116,MTNBJ
1,MOBILE,100,22967393349,SUCCESS,Website,BJ,645a225c4744ee6479066772,Light Browser,104.28.217.116,MTNBJ
2,MOBILE,100,22952734444,TRANSACTION_CANCEL,Website,BJ,645a225c4744ee6479066772,Light Browser,104.28.217.116,MTNBJ
3,MOBILE,100,22963522309,WAITING_CUSTOMER_PAYMENT,Website,BJ,645a225c4744ee6479066772,Light Browser,104.28.217.116,FPZ
4,MOBILE,100,22952734444,TRANSACTION_CANCEL,Website,BJ,645a225c4744ee6479066772,Light Browser,104.28.217.116,MTNBJ


In [ ]:
data['process'] = data['process'].astype('str')
data['statusCode'] = data['statusCode'].astype('str')
data['canal'] = data['canal'].astype('str')
data['country'] = data['country'].astype('str')
data['event'] = data['event'].astype('str')
data['browser'] = data['browser'].astype('str')
data['ip'] = data['ip'].astype('str')
data['agregator'] = data['agregator'].astype('str')

In [ ]:

# Supposons que X contient vos variables explicatives et y votre variable cible
X = data.drop('statusCode', axis=1)  # Variables explicatives
y = data['statusCode']  # Variable cible

# Séparation en train (70%), test (20%) et validation (10%) en fonction de y (statusCode)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42, stratify=y_temp)

# Affichage des tailles des ensembles
print("Taille de l'ensemble d'entraînement (train) :", X_train.shape[0])
print("Taille de l'ensemble de test :", X_test.shape[0])
print("Taille de l'ensemble de validation :", X_val.shape[0])


Taille de l'ensemble d'entraînement (train) : 42585
Taille de l'ensemble de test : 12228
Taille de l'ensemble de validation : 6023


In [ ]:
# Initialiser le modèle CatBoostClassifier
model = CatBoostClassifier(auto_class_weights="Balanced", task_type="GPU", devices='0:1', random_state=42, iterations=150, learning_rate = 0.2, depth = 5 )

# Définir les colonnes catégorielles
cat_features = ['process', 'canal', 'country', 'event', 'browser', 'ip', 'agregator']

# Convertir les colonnes catégorielles en type catégoriel
X_train[cat_features] = X_train[cat_features].astype('category')
X_test[cat_features] = X_test[cat_features].astype('category')

# Créer un objet Pool pour les données d'entraînement et de test
train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)
test_pool = Pool(data=X_test, label=y_test, cat_features=cat_features)

# Entraîner le modèle
model.fit(train_pool)

# Faire des prédictions sur l'ensemble de test
y_pred = model.predict(test_pool)

# Afficher le rapport de classification
print(classification_report(y_test, y_pred))


0:	learn: 1.4866865	total: 15.2ms	remaining: 2.27s
1:	learn: 1.4051171	total: 22.6ms	remaining: 1.67s
2:	learn: 1.3493172	total: 29.3ms	remaining: 1.43s
3:	learn: 1.3110353	total: 35.7ms	remaining: 1.3s
4:	learn: 1.2798734	total: 42.3ms	remaining: 1.23s
5:	learn: 1.2572480	total: 49.2ms	remaining: 1.18s
6:	learn: 1.2354221	total: 56.3ms	remaining: 1.15s
7:	learn: 1.2208404	total: 63.3ms	remaining: 1.12s
8:	learn: 1.2058522	total: 69.9ms	remaining: 1.09s
9:	learn: 1.1944223	total: 76.2ms	remaining: 1.07s
10:	learn: 1.1862974	total: 83.2ms	remaining: 1.05s
11:	learn: 1.1791704	total: 89.2ms	remaining: 1.03s
12:	learn: 1.1688106	total: 95ms	remaining: 1s
13:	learn: 1.1647737	total: 101ms	remaining: 985ms
14:	learn: 1.1605226	total: 108ms	remaining: 968ms
15:	learn: 1.1574787	total: 114ms	remaining: 959ms
16:	learn: 1.1512390	total: 121ms	remaining: 946ms
17:	learn: 1.1477483	total: 127ms	remaining: 931ms
18:	learn: 1.1431376	total: 134ms	remaining: 922ms
19:	learn: 1.1357921	total: 140ms	

In [ ]:
# Sauvegarder l'encodeur LabelEncoder
#joblib.dump(label_encoder, "/content/drive/Othercomputers/Mon ordinateur/DIT MASTER1 2022/MASTER 2/MEMOIRE/data/data/modele/CatBoostClassifier/label_encoder.pkl")

# Sauvegarder le modèle RandomForestClassifier
joblib.dump(model, "/content/drive/Othercomputers/Mon ordinateur/DIT MASTER1 2022/MASTER 2/MEMOIRE/data/data/modele/CatBoostClassifier/CatBoostClassifier.pkl")

['/content/drive/Othercomputers/Mon ordinateur/DIT MASTER1 2022/MASTER 2/MEMOIRE/data/data/modele/CatBoostClassifier/CatBoostClassifier.pkl']

In [ ]:
model.save_model("/content/drive/Othercomputers/Mon ordinateur/DIT MASTER1 2022/MASTER 2/MEMOIRE/data/data/modele/CatBoostClassifier/CatBoostClassifier.cb")


In [ ]:
import gradio as gr
import pandas as pd
from catboost import CatBoostClassifier

# Remplacer avec le chemin de votre modèle entraîné
# Le modèle doit être préalablement sauvegardé via `model.save_model()`

model = CatBoostClassifier()
model.load_model("/content/drive/Othercomputers/Mon ordinateur/DIT MASTER1 2022/MASTER 2/MEMOIRE/data/data/modele/CatBoostClassifier/CatBoostClassifier.cb")  # Remplacez le chemin par le chemin de votre modèle .cb

def predir_model(process, amount, phoneNumber, canal, country, event, browser, ip, agregator):
    input_data = pd.DataFrame([{
        "process": process,
        "amount": amount,
        "phoneNumber": phoneNumber,
        "canal": canal,
        "country": country,
        "event": event,
        "browser": browser,
        "ip": ip,
        "agregator": agregator
    }])
    prediction = model.predict(input_data)
    # Retourner la prédiction sous le format souhaité
    return str(prediction)

iface = gr.Interface(fn=predir_model,
                    inputs=["text", "number", "text", "text", "text", "text", "text", "text", "text"],
                    outputs="text",
                    title="Interface de prédiction CatBoost")

iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c7ac1c23f61c57e0ec.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
